In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from datetime import datetime, timedelta,date
import plotly.express as px
import requests
import io

%matplotlib inline

COVID-19: Spread Progression

In [2]:
url = "http://www.pref.kanagawa.jp/osirase/1369/data/csv/patient.csv"
r = requests.get(url)
df_patient= pd.read_csv(io.BytesIO(r.content),sep=",",encoding="shift-jis")
df_patient.tail()

,発表日,居住地,年代,性別
680,2020-04-17,神奈川県厚木保健福祉事務所管内,20代,男性
681,2020-04-17,神奈川県厚木保健福祉事務所管内,60代,女性
682,2020-04-17,神奈川県厚木保健福祉事務所管内,30代,男性
683,2020-04-17,神奈川県藤沢市,60代,女性
684,2020-04-17,神奈川県藤沢市,50代,男性


In [3]:
#参照する位置情報は事前ダウンロードする。直接CSVリンクがない。
#http://nlftp.mlit.go.jp/cgi-bin/isj/dls/_download_files.cgi
df_lat_long_file = pd.read_csv("kanagawa-14_2018.csv",encoding = "shift-jis")
df_lat_long_file.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,14,神奈川県,14101,横浜市鶴見区,141010001001,本町通一丁目,35.502643,139.680165,0,3
1,14,神奈川県,14101,横浜市鶴見区,141010002001,潮田町一丁目,35.504476,139.682396,0,3
2,14,神奈川県,14101,横浜市鶴見区,141010003000,寛政町,35.501344,139.701084,0,1
3,14,神奈川県,14101,横浜市鶴見区,141010002004,潮田町四丁目,35.503095,139.692805,0,3
4,14,神奈川県,14101,横浜市鶴見区,141010004000,東寺尾東台,35.503422,139.663808,0,1


In [4]:
df_patient['居住地'].unique()

array(['神奈川県', '神奈川県相模原市', '神奈川県横浜市', '神奈川県鎌倉保健福祉事務所管内',
       '神奈川県厚木保健福祉事務所管内', '神奈川県小田原保健福祉事務所管内', '神奈川県平塚保健福祉事務所管内',
       '神奈川県藤沢市', '神奈川県川崎市', '神奈川県茅ヶ崎市保健所管内', '神奈川県横須賀市', 'スペイン（横浜市発表）',
       '国外（川崎市発表）', '川崎市外（川崎市発表）', '神奈川県川崎市外（川崎市発表）', '神奈川県茅ケ崎市保健所管内及び都内',
       '神奈川県茅ケ崎市保健所管内', '横浜市内', '横須賀市内', '川崎市内', '相模原市内', '藤沢市内', '神奈川県内',
       '川崎市外', '横浜市外', '東京都\u3000'], dtype=object)

In [5]:
#df_data["発表日"] = pd.to_datetime( df_data["発表日"]).dt.strftime('%m/%d/%Y')

"市外","（〇〇市発表）"の居住地は削除して、居住地検索するときに影響しないようにする。

In [6]:
df_data = df_patient[~df_patient['居住地'].str.contains("外")]
df_data = df_data[~df_data['居住地'].str.contains('\（')]
df_data

,発表日,居住地,年代,性別
0,2020-01-16,神奈川県,30代,男性
1,2020-02-11,神奈川県,50代,男性
2,2020-02-13,神奈川県,80代,女性
3,2020-02-14,神奈川県,30代,男性
4,2020-02-17,神奈川県相模原市,40代,女性
...,...,...,...,...
680,2020-04-17,神奈川県厚木保健福祉事務所管内,20代,男性
681,2020-04-17,神奈川県厚木保健福祉事務所管内,60代,女性
682,2020-04-17,神奈川県厚木保健福祉事務所管内,30代,男性
683,2020-04-17,神奈川県藤沢市,60代,女性


In [7]:
df_data['居住地'].unique()

array(['神奈川県', '神奈川県相模原市', '神奈川県横浜市', '神奈川県鎌倉保健福祉事務所管内',
       '神奈川県厚木保健福祉事務所管内', '神奈川県小田原保健福祉事務所管内', '神奈川県平塚保健福祉事務所管内',
       '神奈川県藤沢市', '神奈川県川崎市', '神奈川県茅ヶ崎市保健所管内', '神奈川県横須賀市',
       '神奈川県茅ケ崎市保健所管内及び都内', '神奈川県茅ケ崎市保健所管内', '横浜市内', '横須賀市内', '川崎市内',
       '相模原市内', '藤沢市内', '神奈川県内', '東京都\u3000'], dtype=object)

In [8]:
df_lat_log = df_lat_long_file[["市区町村名","緯度","経度"]]
#市レベルまでの精度でよいので区までのデータ列を一つずつ代表だけ抽出
df_lat_log.drop_duplicates(subset = "市区町村名",inplace = True)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
cut_after_shi = lambda s: s.split("市")[0]
cut_after_gun = lambda s: s.split("郡")[0]

In [15]:
df_lat_log.loc[:,"市区町村名"] = df_lat_log["市区町村名"].apply(cut_after_shi)
df_lat_log.loc[:,"市区町村名"] = df_lat_log["市区町村名"].apply(cut_after_gun)
#代表となる地図座標は各市の区毎座標の平均値をとる。これが地図上の円の円心となる。
df_lat_log = df_lat_log.groupby("市区町村名").mean().reset_index()
df_lat_log

,市区町村名,緯度,経度
0,三浦,35.215733,139.610838
1,中,35.304284,139.277916
2,伊勢原,35.427170,139.246068
3,南足柄,35.293640,139.133070
4,厚木,35.415655,139.352386
5,大和,35.507309,139.433700
6,小田原,35.248714,139.123280
7,川崎,35.576587,139.616441
8,平塚,35.320696,139.370693
9,座間,35.501027,139.414615


In [16]:
#Dictionary型にして、Map表示の準備をする。
dict_lat_log = dict(zip(df_lat_log['市区町村名'] , zip(df_lat_log['緯度'],df_lat_log['経度'])))
dict_lat_log

{'三浦': (35.2157335, 139.6108375),
 '中': (35.304284499999994, 139.277916),
 '伊勢原': (35.427170000000004, 139.24606799999998),
 '南足柄': (35.29364, 139.13307),
 '厚木': (35.415655, 139.35238600000002),
 '大和': (35.507309, 139.4337),
 '小田原': (35.248714, 139.12328),
 '川崎': (35.57658728571429, 139.61644142857142),
 '平塚': (35.320696000000005, 139.370693),
 '座間': (35.501027, 139.414615),
 '愛甲': (35.500660499999995, 139.2996145),
 '横浜': (35.454208111111114, 139.57947977777778),
 '横須賀': (35.206801, 139.61791599999998),
 '海老名': (35.449496, 139.382276),
 '相模原': (35.541925666666664, 139.37690433333333),
 '秦野': (35.361692, 139.24338999999998),
 '綾瀬': (35.420321, 139.442378),
 '茅ヶ崎': (35.328485, 139.375121),
 '藤沢': (35.335188, 139.49716899999999),
 '足柄上': (35.357312, 139.13808840000002),
 '足柄下': (35.19510666666667, 139.08163133333332),
 '逗子': (35.281544, 139.610675),
 '鎌倉': (35.320847, 139.497922),
 '高座': (35.365845, 139.37102)}

In [17]:
df_data['居住地'] = df_data['居住地'].str.replace('茅ケ崎','茅ヶ崎')

In [18]:
for index in dict_lat_log.keys() :
    change_add = lambda s: index if index in s else s 
    df_data['居住地'] = df_data['居住地'].apply(change_add)

In [19]:
df_data= df_data.groupby(['居住地']).count()


In [20]:
df_data = df_data.reset_index()
df_data

,居住地,発表日,年代,性別
0,厚木,55,55,55
1,小田原,18,18,18
2,川崎,145,145,145
3,平塚,41,41,41
4,東京都,1,1,1
5,横浜,215,129,129
6,横須賀,32,32,32
7,相模原,55,55,55
8,神奈川県,4,4,4
9,神奈川県内,1,1,1


In [21]:
df_data["Lat"] = ""
df_data["Long"] = ""
for index in dict_lat_log.keys() :
    df_data.loc[df_data['居住地'].str.contains(index),"Lat"] = dict_lat_log[index][0]
    df_data.loc[df_data['居住地'].str.contains(index),"Long"] = dict_lat_log[index][1]

In [22]:
#辞書から市名が見つからなかったのはそもそも表示ができないので対象にしない
df_data = df_data[~(df_data["Lat"] == "")]
df_data = df_data.reset_index(drop = True)
df_data

,居住地,発表日,年代,性別,Lat,Long
0,厚木,55,55,55,35.4157,139.352
1,小田原,18,18,18,35.2487,139.123
2,川崎,145,145,145,35.5766,139.616
3,平塚,41,41,41,35.3207,139.371
4,横浜,215,129,129,35.4542,139.579
5,横須賀,32,32,32,35.2068,139.618
6,相模原,55,55,55,35.5419,139.377
7,茅ヶ崎,19,19,19,35.3285,139.375
8,藤沢,41,41,41,35.3352,139.497
9,鎌倉,46,46,46,35.3208,139.498


Todo: 累積データにした方がよい

In [42]:
#地図の真ん中に'横浜': (35.454208111111114, 139.57947977777778)を置く
kanagawa = folium.Map(location=[35.454208111111114, 139.57947977777778], zoom_start=9,max_zoom=12,min_zoom=4,height=500,width="80%")
for i in range(0,len(df_data.index)):
    folium.Circle(
        location=[df_data.iloc[i]['Lat'], df_data.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+df_data.iloc[i]["居住地"].encode("shift-jis")+"</h5>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>Confirmed: "+str(df_data.iloc[i]['年代'])+"</li>"+
#        "<li>Active:   "+str(df_india.iloc[i]['active'])+"</li>"+
#        "<li>Recovered:   "+str(df_india.iloc[i]['recovered'])+"</li>"+
#        "<li>Deaths:   "+str(df_india.iloc[i]['deaths'])+"</li>"+        
        "</ul>"
        ,
        radius=(int(np.log2(df_data.iloc[i]['年代']+1)))*1000,
        color='#ff6600',
        fill_color='#ff8533',
        fill=True).add_to(kanagawa)

kanagawa

TypeError: can only concatenate str (not "bytes") to str